In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy  

In [31]:
!pip install --upgrade syft


  Found existing installation: syft 0.1.18
    Uninstalling syft-0.1.18:
      Successfully uninstalled syft-0.1.18


In [19]:
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client") 
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 

W0909 21:21:34.768234 16484 hook.py:97] Torch was already hooked... skipping hooking process


In [20]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 200
        self.epochs = 10
        self.lr = 0.001 # learning rate
        self.log_interval = 100

args = Arguments()

Here, we assume that the server will have some data to get a first instance model generated.

In [21]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

In [22]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

# Convert to integers and privately share the dataset
private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_prec().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_prec().share(alice, bob, crypto_provider=crypto_provider)
    ))

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [24]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader)))

In [25]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]
Train Epoch: 1 [6400/60032 (11%)]
Train Epoch: 1 [12800/60032 (21%)]
Train Epoch: 1 [19200/60032 (32%)]
Train Epoch: 1 [25600/60032 (43%)]
Train Epoch: 1 [32000/60032 (53%)]
Train Epoch: 1 [38400/60032 (64%)]
Train Epoch: 1 [44800/60032 (75%)]
Train Epoch: 1 [51200/60032 (85%)]
Train Epoch: 1 [57600/60032 (96%)]
Train Epoch: 2 [0/60032 (0%)]
Train Epoch: 2 [6400/60032 (11%)]
Train Epoch: 2 [12800/60032 (21%)]
Train Epoch: 2 [19200/60032 (32%)]
Train Epoch: 2 [25600/60032 (43%)]
Train Epoch: 2 [32000/60032 (53%)]
Train Epoch: 2 [38400/60032 (64%)]
Train Epoch: 2 [44800/60032 (75%)]
Train Epoch: 2 [51200/60032 (85%)]
Train Epoch: 2 [57600/60032 (96%)]
Train Epoch: 3 [0/60032 (0%)]
Train Epoch: 3 [6400/60032 (11%)]
Train Epoch: 3 [12800/60032 (21%)]
Train Epoch: 3 [19200/60032 (32%)]
Train Epoch: 3 [25600/60032 (43%)]
Train Epoch: 3 [32000/60032 (53%)]
Train Epoch: 3 [38400/60032 (64%)]
Train Epoch: 3 [44800/60032 (75%)]
Train Epoch: 3 [51200/60032 (85%)]
Tra

In [26]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [29]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
            
            n_correct = n_correct_priv.copy()
            n_c1 = n_correct.get()
            n_c2 = n_c1.float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))

In [30]:
test(args, model, private_test_loader)


AttributeError: 'int' object has no attribute 'is_wrapper'